# Training process


* Resnet 을 기본으로 함
* 첫번째 필터의 필터사이즈를 7 -> 10 
* BN 과 dropout을 같이 사용해봄 
* 훈련데이터셋을 datagen.flow 로 두배증식후 다시 imagedatagenerator 사용 
* LR 0.003 훈련중 saddle point 에 들어선거 같아 벗어나기 위해 0.05로 변경 (Loss 더 상승)
* 다시 LR 0.003로 변경함

In [1]:
from keras.utils import np_utils
from tensorflow.keras import Sequential, models, layers
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation, BatchNormalization, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications import ResNet50
from matplotlib.image import imread
from cv2 import imread
from tqdm import tqdm
from PIL import Image
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
import imutils
import zipfile

seed = 0

Using TensorFlow backend.


In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
def macro_f1(y, y_hat, thresh=0.5):
    """Compute the macro F1-score on a batch of observations (average F1 across labels)

    Args:
        y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        thresh: probability value above which we predict positive

    Returns:
        macro_f1 (scalar Tensor): value of macro F1 for the batch
    """
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [31]:
train_data = pd.read_csv('train_data.csv')
validation_data = pd.read_csv('validation_data.csv')
test_data = pd.read_csv('test_data.csv')
target = pd.read_csv('data/dirty_mnist_2nd_answer.csv',index_col='index')
target_names = target.columns

In [5]:
newtrain_data = pd.read_csv('newtrain.csv')
newvalidation_data = pd.read_csv('newvalidation.csv')
test_data = pd.read_csv('test_data.csv')
newtarget = pd.read_csv('data/newTarget.csv',index_col='index')
target_names = newtarget.columns

In [6]:
LR = .003 # Keep it small when transfer learning
EPOCHS = 300
IMG_SIZE = 256 # Specify height and width of image to match the input format of the model
CHANNELS = 1 # Keep RGB color channels to match the input format of the model
# CHANNELS change by 1 from 3
N_LABELS = 26

In [7]:
BATCH_SIZE = 48
#BATCH_SIZE = 256 # Big enough to measure an F1-score
AUTOTUNE = tf.data.experimental.AUTOTUNE # Adapt preprocessing and prefetching dynamically to reduce GPU and CPU idle time
SHUFFLE_BUFFER_SIZE = 1024 # Shuffle the training data by a chunck of 1024 observations

In [8]:
# 정규화변경 featurewise_std_normalization = True 로

generator = ImageDataGenerator(rescale=1./255)

train_datagen = ImageDataGenerator(
   
    
    rescale=1./255, 
    
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='constant',
    cval = 0)

train_generator = train_datagen.flow_from_dataframe(
    newtrain_data,
    'data/train_dirty_mnist_2nd/',
    x_col='filename',
    y_col=list(newtrain_data.columns[1:]),
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='raw',
    color_mode='grayscale',
    batch_size=BATCH_SIZE
)

validation_generator = generator.flow_from_dataframe(
    newvalidation_data,
    'data/train_dirty_mnist_2nd/',
    x_col='filename',
    y_col=list(newvalidation_data.columns[1:]),
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='raw',
    color_mode='grayscale',
    batch_size=BATCH_SIZE
)

test_generator = generator.flow_from_dataframe(
    test_data,
    'data/test_dirty_mnist_2nd/',
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    shuffle=False
)

Found 80000 validated image filenames.
Found 20000 validated image filenames.
Found 5000 validated image filenames.


In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    vertical_flip= True,
    horizontal_flip=True,
    fill_mode='constant',
    cval = 1)

img = load_img('data/train_dirty_mnist_2nd/00000.png')  # PIL 이미지
x = img_to_array(img)  # (3, 150, 150) 크기의 NumPy 배열
x = x.reshape((1,) + x.shape)  # (1, 3, 150, 150) 크기의 NumPy 배열

# 아래 .flow() 함수는 임의 변환된 이미지를 배치 단위로 생성해서
# 지정된 preview/ 폴더에 저장합니다.
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview_update', save_prefix='emnist', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # 이미지 20장을 생성하고 마칩니다

SyntaxError: invalid syntax (<ipython-input-8-46fc650e7766>, line 10)

# wresV7(normal resnet, filter16)

* 모델을 단순화 해서 피팅하려면 어떻게 해야할까?

* 0213 edit - augmentation (val_loss = 0.5791) 
* 0214 edit - 첫번째 레이어의 필터의 크기를 늘려 10 by 10 stride 2좀더 큰 패턴을 찾아봄
* 0214 edit - 데이터의 분포가 중심에 몰려있어 첫 레이어에서 패딩을 하지않음 (val_loss = 0.38) 점수 0.78
* 0214 edit - 첫번째 필터 10,10 stride 2 & 각레이어 세번째 마다 dropout(.2) 추가 (val_loss = 0.35) 점수 0.81
* 0218 edit - 첫번째 필터 16,16 stride 2 (val_loss = 0.31) 점수 0.80
* 0219 edit - 5층 제거 후 마지막층 filter 512 로 변경
* 0221 edit - 첫층 필터 64 -> 256 , 드랍아웃 0.8
* 0225 edit - BN 삭제


In [9]:
from tensorflow.keras import Input
from tensorflow.keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add
from tensorflow.keras.layers import AveragePooling2D, Dropout
from tensorflow.keras.models import Model, load_model

In [10]:
# number of classes
K = 26
 
 
input_tensor = Input(shape=(256, 256, 1), dtype='float32', name='input')
 
 
def conv1_layer(x):    
    # og - x = Conv2D(64, (10, 10), strides=(2, 2))(x)
    x = Conv2D(64, (10, 10), strides=(2, 2))(x) # edit 0221
    
    x = BatchNormalization()(x)
    
    x = Activation('relu')(x)
    #x = Dropout(.6)(x) #edit 0221
    
    return x   
 
    
    
def conv2_layer(x):         
    x = MaxPooling2D((3, 3), 2)(x)     
 
    shortcut = x
 
    for i in range(3):
        if (i == 0):
            x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
            
            x = BatchNormalization()(x)
            
            x = Activation('relu')(x)
 
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(shortcut)  
        
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)
 
            x = Add()([x, shortcut])
            x = Activation('relu')(x)
       
        
            
            shortcut = x
 
        else:
            x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)
            
            x = BatchNormalization()(x)
            
            x = Activation('relu')(x)
            
            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
            
            x = BatchNormalization()(x)
            
            x = Activation('relu')(x)
 
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
    
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])   
            x = Activation('relu')(x)  
            
            
            shortcut = x        
    
    return x
 
 
 
def conv3_layer(x):        
    shortcut = x    
    
    for i in range(4):     
        if(i == 0):            
            x = Conv2D(128, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)            
 
            x = Add()([x, shortcut])    
            x = Activation('relu')(x)    
           
            
            shortcut = x              
        
        else:
            x = Conv2D(128, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])     
            x = Activation('relu')(x)
            
            
            shortcut = x      
            
    return x
 

def conv3_layer_exdrop(x):        
    shortcut = x    
    
    for i in range(4):     
        if(i == 0):            
            x = Conv2D(128, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)            
 
            x = Add()([x, shortcut])    
            x = Activation('relu')(x)    
           
            
            shortcut = x              
        
        else:
            x = Conv2D(128, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])     
            x = Activation('relu')(x)
            
            
            shortcut = x      
            
    return x
     
    
 
def conv4_layer(x):
    shortcut = x        
  
    for i in range(6):     
        if(i == 0):            
            x = Conv2D(256, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)
 
            x = Add()([x, shortcut]) 
            x = Activation('relu')(x)
            
            
            shortcut = x               
        
        else:
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])    
            x = Activation('relu')(x)
            
            
            shortcut = x      
 
    return x
 
def conv5_layer(x):
    shortcut = x        
  
    for i in range(6): 
        if(i == 0):            
            x = Conv2D(256, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)
 
            x = Add()([x, shortcut]) 
            x = Activation('relu')(x)
            
            
            shortcut = x               
        
        else:
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])    
            x = Activation('relu')(x)
           
            
            shortcut = x      
 
    return x
 
def conv6_layer(x):
    shortcut = x        
  
    for i in range(6):     
        if(i == 0):            
            x = Conv2D(256, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)
 
            x = Add()([x, shortcut]) 
            x = Activation('relu')(x)
            
            
            shortcut = x               
        
        else:
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])    
            x = Activation('relu')(x)
           
            
            shortcut = x      
 
    return x
  



 
x = conv1_layer(input_tensor)
x = conv2_layer(x)
x = conv3_layer(x)
#x = conv3_layer_exdrop(x)
x = conv4_layer(x)
x = conv5_layer(x)
#x = conv6_layer(x)


 
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)

output_tensor = Dense(K, activation='sigmoid')(x)

In [11]:
Wlayer = Model(input_tensor, output_tensor)
Wlayer.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 124, 124, 64) 6464        input[0][0]                      
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 124, 124, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 124, 124, 64) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

# Load savedmodel

In [12]:
Wlayer.load_weights('Mnistmodel/resNoBN_15-0.2763.h5')

# Complie

In [13]:
#LR = 0.003

In [14]:
MODEL_DIR = './Mnistmodel/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath = './Mnistmodel/resNoBN_{epoch:02d}-{val_loss:.4f}.h5'
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss',
                              verbose=1, save_best_only=True)

stop = EarlyStopping(patience=100)

callbacks = [stop, checkpointer]

In [15]:
Wlayer.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
              metrics=[macro_f1])

# Fitting

* 지금 모델에서 단순화해서 테스트셋에 피팅 하려면 어떻게 해야할까

In [16]:
history = Wlayer.fit_generator(train_generator, epochs=EPOCHS,
                              validation_data=validation_generator, callbacks=callbacks,shuffle=True
                              )

C:\Users\USER\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1876: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/300
1667/1667 [==============================] - 504s 292ms/step - loss: 0.3097 - macro_f1: 0.8423 - val_loss: 0.3321 - val_macro_f1: 0.8417

Epoch 00001: val_loss improved from inf to 0.33207, saving model to ./Mnistmodel\resNoBN_01-0.3321.h5
Epoch 2/300
1667/1667 [==============================] - 478s 287ms/step - loss: 0.3078 - macro_f1: 0.8432 - val_loss: 0.2838 - val_macro_f1: 0.8680

Epoch 00002: val_loss improved from 0.33207 to 0.28384, saving model to ./Mnistmodel\resNoBN_02-0.2838.h5
Epoch 3/300
1667/1667 [==============================] - 480s 288ms/step - loss: 0.3062 - macro_f1: 0.8448 - val_loss: 0.2730 - val_macro_f1: 0.8739

Epoch 00003: val_loss improved from 0.28384 to 0.27296, saving model to ./Mnistmodel\resNoBN_03-0.2730.h5
Epoch 4/300
1667/1667 [==============================] - 493s 296ms/step - loss: 0.3053 - macro_f1: 0.8452 - val_loss: 0.2932 - val_macro_f1: 0.8687

Epoch 00004: val_loss did not improve from 0.27296
Epoch 5/300
1667/1667 [============

KeyboardInterrupt: 

In [33]:
Wlayer.evaluate(validation_generator)

417/417 [==============================] - 36s 85ms/step - loss: 0.2824 - macro_f1: 0.8664


[0.2823958396911621, 0.8664236068725586]

In [17]:
predict = Wlayer.predict_generator(test_generator)

C:\Users\USER\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1937: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [18]:
submit = pd.DataFrame(predict, columns=newtarget.columns)
submission = pd.read_csv('data/sample_submission.csv')

for i in submit.columns:
    for j in range(len(submit)):
        if submit[i][j] > 0.5:
            submit[i][j] = int(1)
        else:
            submit[i][j] = int(0)


submission = pd.DataFrame(submission['index'])
for i in submit.columns:
    submission[i] = submit[i]

In [19]:
submission.to_csv('submission03013.csv', index = False)

In [20]:
submission.head()

,index,a,b,c,d,e,f,g,h,i,...,q,r,s,t,u,v,w,x,y,z
0,50000,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0
1,50001,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,50002,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,50003,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,50004,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0


In [23]:
submission.iloc[:,1:].sum(axis=1).mean()

11.3302001953125